# The executive and processes in the Solution Building Library

This example notebook demonstrates how to use the executive and how to create processes in the Solution Building Library.

<div class="alert alert-info">

**Important**

This notebook requires a running Flowstate solution to connect to. To start a solution:

1. Navigate to [flowstate.intrinsic.ai](https://flowstate.intrinsic.ai/) and sign in
   using your registered Flowstate account.

1. Do **one** of the following:
    - Create a new solution:
        1. Click "Create new solution" and choose "From an example".
        1. Select `pick_and_place:pick_and_place_module2`
        1. Click "Create".
    - Or open an existing solution that was created from the `pick_and_place:pick_and_place_module2` example:
        1. Hover over the solution in the list.
        1. Click "Open solution" or "Start solution".

1. Recommended: Keep the browser tab with the Flowstate solution editor open to watch the effect of notebook actions such as running a skill. You can simultaneously interact with the solution through the web UI and the notebook.

</div>

## Connect to solution

Let's start with the typical preamble:

- Import the relevant modules.
- Connect to the deployed solution.
- Define some shortcut variables for convenience.

In [ ]:
from intrinsic.solutions import deployments
from intrinsic.solutions import behavior_tree as bt

solution = deployments.connect_to_selected_solution()

executive = solution.executive
skills = solution.skills
world = solution.world
simulator = solution.simulator

## Executive

The `executive` is the main entrypoint for running skills and processes in the solution. To demonstrate its usage we first create a few sample skills so that we have *something* to execute. The following example skills here move the robot, so it will be easy to see the effect of running them in the Flowstate solution editor. Creating and parameterizing skills is explained in detail in the "skills" example notebook.

In [ ]:
move_robot = skills.ai.intrinsic.move_robot

# Moves the robot to the 'home' pose.
move_skill_1 = move_robot(
    motion_segments=[
        move_robot.intrinsic_proto.skills.MotionSegment(
            joint_position=world.robot.joint_configurations.home,
            motion_type=move_robot.intrinsic_proto.skills.MotionSegment.MotionType.JOINT,
        )
    ],
    arm_part=world.robot,
)

# Moves the robot to 'view_pose_left'.
move_skill_2 = move_robot(
    motion_segments=[
        move_robot.intrinsic_proto.skills.MotionSegment(
            joint_position=world.robot.joint_configurations.view_pose_left,
            motion_type=move_robot.intrinsic_proto.skills.MotionSegment.MotionType.JOINT,
        )
    ],
    arm_part=world.robot,
)

### Synchronous execution

You can run a single skill like this:

In [ ]:
executive.run(move_skill_2)

 While the cell is executing, you can watch the robot move in the Flowstate solution editor (make sure that the 3D scene view is set to the "Execute" tab).

 You can also run a sequence of skills:

In [ ]:
executive.run([move_skill_1, move_skill_2])

After the execution has finished, you can choose "Process" --> "Load" --> "From executive" from the menu to view, edit or replay the last executed process in the process editor. Because we have passed skill instances directly to `executive.run()`, observe that the skills are unnamed and that the process is called `(untitled)`. This is useful for testing, but usually you should wrap skill instances in nodes of a `BehaviorTree`. The `BehaviorTree` class represents processes in the Solution Building Library:

In [ ]:
from intrinsic.solutions import behavior_tree as bt

tree = bt.BehaviorTree(
    name="My first behavior tree",
    root=bt.Sequence([
        bt.Task(action=move_skill_1, name="Some move"),
        bt.Task(action=move_skill_2, name="Another move"),
    ]),
)

executive.run(tree)

This gives you the same capabilities as the process editor of the Flowstate solution editor. E.g., it allows for naming nodes and opens up the possibility to use flow control nodes such as `Branch` or `Loop`. Here is one more example tree with an endless loop which will be useful in the following:

In [ ]:
tree_with_loop = bt.BehaviorTree(
    name="Move back and forth forever",
    root=bt.Loop(
        while_condition=bt.Blackboard("true"),
        do_child=bt.Sequence([
            bt.Task(action=move_skill_1, name="Some move"),
            bt.Task(action=move_skill_2, name="Another move"),
        ]),
    ),
)

You can find more complex behavior trees in the other example notebooks.

### Asynchronous execution

Executing a behavior tree can take a while and `executive.run()` will block until the execution has finished. If you want to do something during execution, you can use `run_async`. E.g., you can observe the different state transitions inside the executive:

In [ ]:
from intrinsic.executive.proto import behavior_tree_pb2


def print_executive_state():
    print(
        "Executive state:",
        behavior_tree_pb2.BehaviorTree.State.Name(
            executive.operation.metadata.behavior_tree_state
        ),
    )


def print_is_succeeded():
    print(
        "Is succeeded:",
        executive.operation.metadata.behavior_tree_state
        == behavior_tree_pb2.BehaviorTree.SUCCEEDED,
    )


print_executive_state()

executive.run_async(tree)
print_executive_state()

# ... do something here ...

executive.block_until_completed()
print_executive_state()
print_is_succeeded()

Note that `executive.operation.metadata` is a "Protocol Buffer" (proto) message. You can find all about using protos in Python in the official [Python Generated Code Guide]( https://protobuf.dev/reference/python/python-generated/).

Here you can see the first transition to `RUNNING` and, after calling `executive.block_until_completed()`, you can see the transition to `SUCCEEDED`.

You can also interrupt the execution of the behavior tree and resume it. This is done by using `executive.suspend()` and `executive.resume()` as follows:

In [ ]:
# This tree has an endless loop and will not finish on its own.
executive.run_async(tree_with_loop)
print_executive_state()

# ... do something here ...

executive.suspend()
print_executive_state()

# ... do something here ...

executive.resume()
print_executive_state()

`executive.suspend()` waits for the currently running skills in the behavior tree to finish and then stops the executive. When `executive.resume()` is called the next skill gets executed.
Calling `executive.suspend()` while an action is running leads to the executive being in state `SUSPENDING` until the execution of the skill has finished.
Only afterwards does the executive transition to `SUSPENDED` and therefore succeeds the `executive.suspend()` operation and continues with the program.

If your executive ends up in `FAILED` state the errors are displayed automatically inside the notebook.

You can cancel execution immediately (without the option to resume) by using `executive.cancel()` or `executive.cancel_async()`.



In [ ]:
executive.cancel()
print_executive_state()

Calling `executive.cancel()` while an action is running leads to the executive being in state `CANCELING` until the running skill finishes cancelling (or, if it does not support cancellation, finishes execution as usual). Afterwards, the executive ends in either the state `CANCELED` (if the cancellation was processed) or in `SUCCEEDED`/`FAILED` (if it finished in success/failure before processing the cancellation).

## Resetting

Various components of the solution can be reset separately from each other.

If you have unsaved world modifications as a result of running certain skills or because you edited the belief world you can restore the belief world to its last saved state like this:

In [ ]:
world.reset()

For more ways to interact with the belief world see the `003_world.ipynb` example.

You can reset the simulation manually which is the same as clicking **Reset** in the **Simulator** tab of the [workcell designer](https://developers.intrinsic.ai/guides/workcell_design/workcell_overview) of the Flowstate solution editor. The simulation state will be reset to the state of the **Belief** world.

In [ ]:
simulator.reset()

## Processes

Processes are represented as assets in Flowstate - similar to skills or services. Process assets which are installed in a solution can be accessed in the Solution Building Library through `solution.processes`.

In [ ]:
processes = solution.processes

Process assets are represented by `BehaviorTree` objects. However, a `BehaviorTree` instance by default only has a display name and no further asset metadata. It is a local, "anonymous" process which can be sent directly to the executive for execution as we have done above. To be able to save the process as a Process asset to the solution you need to first setup the asset metadata (at least the required fields).

In [ ]:
tree = bt.BehaviorTree(
    name="My first process",
    root=bt.Sequence([
        bt.Task(action=move_skill_1, name="Some move"),
        bt.Task(action=move_skill_2, name="Another move"),
    ]),
)
tree.set_asset_metadata(
    id="com.myorg.my_first_process",
    vendor="MyOrg",
)

# Save 'tree' as Process asset with the ID 'com.myorg.my_first_process'
processes.save(tree)

Now you can verify that your process was saved by listing all processes in the solution. `solution.processes` supports dict-like read access so you can use `keys()`, `items()` and `values()` on it. For example, the following printout should include the ID of our newly saved process.

In [ ]:
print(processes.keys())


A Process asset can also be loaded from the solution. This yields an instance of `BehaviorTree` representing the process. For example, we can load a copy of a saved process and save that copy under a different asset ID.

In [ ]:
other_tree = processes['com.myorg.my_first_process']
# Change asset metadata to a different ID
other_tree.set_asset_metadata(id='com.myorg.my_second_process', vendor='MyOrg')
processes.save(other_tree)

# Should include 'com.myorg.my_first_process' and 'com.myorg.my_second_process'
print(processes.keys())

We can also delete a Process asset from the solution.

In [ ]:
del solution.processes['com.myorg.my_second_process']

# Should print only 'com.myorg.my_first_process'
print(processes.keys())

### Advanced

Last but not least, you can export the behavior tree of a process as a proto file. This is only required for some advandced use cases, e.g., if you want to use your behavior tree as input to Bazel build rules from the Intrinsic SDK.

In [ ]:
from google.protobuf import text_format

file_content = text_format.MessageToString(tree.proto)
with open('tree.txtpb', 'w') as file:
    file.write(file_content)